In [1]:
# Dependencies
# ----------------------------------
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
# Dependencies
# ----------------------------------
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
conn = engine.connect()

# load the data into a Pandas DataFrame
emmission_db = pd.read_csv('Resources/rawdata.csv')
# write the data to a sqlite table
emmission_db.to_sql('emmission_db', conn, if_exists='append', index = False)

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
# import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/country_db.sqlite")

In [5]:
# View all of the classes that automap found

# Declare a Base using 'automap_base()'
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

[]

In [8]:
# Save references to each table
# Assign the 'measurement' to a variable called 'Measurement'
Country = Base.classes.Country

AttributeError: Country

In [ ]:
# Create a session
session = Session(engine)

In [ ]:
# Display the class 'measurement' columns and data in first rows in dictionary format
session.query(measurement).first().__dict__

In [ ]:
# Display the class 'station' columns and data in first rows in dictionary format
session.query(station).first().__dict__

In [ ]:
# link sqlite format to dataframe
conn = engine.connect()
hawaii_measurement = pd.read_sql('SELECT * FROM measurement', conn)
print (hawaii_measurement)

In [ ]:
# link sqlite format to dataframe
hawaii_station = engine.execute('SELECT * FROM station')
# reflect the tables
for record in hawaii_station:
    print(record)

In [ ]:
conn = engine.connect()
hawaii_station = pd.read_sql('SELECT * FROM station', conn)
hawaii_station.head(5)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
conn = engine.connect()
most_recent_date = pd.read_sql('SELECT * FROM measurement ORDER BY date DESC LIMIT 1', conn)
print(f'The most recent date in the data set is')
print(f'{most_recent_date.date}')

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database. And Select only the "date" and "prcp" values.
date_prcp_df = pd.read_sql("SELECT date, prcp FROM measurement WHERE date <= '2017-08-23' AND date >= '2016-08-23'", conn)

In [ ]:
# Load the query results into a Pandas DataFrame, and set the index to the "date" column.
print(date_prcp_df)
date_prcp_df.set_index('date')

In [ ]:
# Sort the DataFrame values by "date".
df = date_prcp_df.set_index('date').sort_index(ascending=True)
df_max = date_prcp_df.groupby(by='date').max()
df_max

In [ ]:
xpoints = df_max.index
ypoints = df_max['prcp']

plt.plot(xpoints, ypoints)
plt.xlabel('Date')
plt.ylabel('mm')
plt.title('precipitation in 08/2016-08/2017')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM
df_mean = df.groupby(by=["date"])["prcp"].mean()
df_median = df.groupby(by=["date"])["prcp"].median()
df_var = df.groupby(by=["date"])["prcp"].var()
df_std = df.groupby(by=["date"])["prcp"].std()
df_sem = df.groupby(by=["date"])["prcp"].sem()
df_sum = df.groupby(by=['date']).sum()

In [ ]:
# Assemble the resulting series into a single summary dataframe.
summary_df = {}
summary_df = pd.DataFrame({
    "mean": df_mean,
    "median": df_median,
    "var": df_var,
    "std": df_std,
    "sem": df_sem
})
summary_df

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_Stations= session.query(measurement.station).group_by('station').count()
print(f'The total number stations are {total_Stations}')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_count = pd.read_sql("SELECT * FROM measurement", conn)

In [ ]:
station_count_1 = station_count.groupby(by='station').count()
station_activate = station_count_1.sort_values(by=['prcp'],ascending=False).head(1)
print(f'The most activate station is {station_activate.index}')

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
min_tobs = session.query(func.min(measurement.tobs)).filter(measurement.station == 'USC00519281').all()
max_tobs = session.query(func.max(measurement.tobs)).filter(measurement.station == 'USC00519281').all()
avg_tobs = session.query(func.avg(measurement.tobs)).filter(measurement.station == 'USC00519281').all()

print(f'The most active station id is USC00519281.')
print(f'Its lowest temperature is {min_tobs}.')
print(f'Its highest temperature is {max_tobs}.')
print(f'Its average temperature is {avg_tobs}.')

In [ ]:
tobs_min = session.query(func.min(measurement.tobs)).filter(measurement.date >= '2016-01-01', measurement.date <= '2017-01-01').all()
tobs_max = session.query(func.max(measurement.tobs)).filter(measurement.date >= '2016-01-01', measurement.date <= '2017-01-01').all()
tobs_avg = session.query(func.avg(measurement.tobs)).filter(measurement.date >= '2016-01-01', measurement.date <= '2017-01-01').all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
date_tobs_df = pd.read_sql("SELECT date, tobs FROM measurement WHERE date <= '2017-08-23' AND date >= '2016-08-23' AND station = 'USC00519281'", conn)
date_tobs_df

In [ ]:
x = np.array(date_tobs_df.tobs)
plt.hist(x)

plt.xlabel("Temperature")
plt.ylabel('Frequency')
plt.legend('tobs')

plt.show()

# Close session

In [ ]:
# Close Session
session.close()